<a href="https://colab.research.google.com/github/gc2321/ztm_pytorch/blob/main/miRNA_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score


In [9]:
print(torch.__version__)

2.5.0+cu121


In [10]:
torch.cuda.is_available()

True

In [11]:
# set uo device agnoistic code
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
# count number of devices
torch.cuda.device_count()

1


## Load Data

In [13]:
# mount the colab
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
import fileinput
filePath = "/content/gdrive/MyDrive/neural_data/miRNA_lung.csv"

In [21]:
data = pd.read_csv(filePath)

In [23]:
data.head()

,depmap_id,cell_line_display_name,lineage_1,lineage_2,lineage_3,lineage_5,lineage_6,lineage_4,MIMAT0000062,MIMAT0000063,...,MIMAT0002192,MIMAT0002191,MIMAT0002190,MIMAT0002189,MIMAT0002188,MIMAT0002187,MIMAT0002186,MIMAT0002185,MIMAT0010151,MIMAT0010150
0,ACH-000382,CORL24,Lung,Lung Neuroendocrine Tumor,Small Cell Lung Cancer,NaN,NaN,NaN,14.735474,12.091720,...,6.113534,6.609991,6.308885,6.262471,6.397461,5.966938,5.907852,6.572435,6.230741,6.910133
1,ACH-000187,CORL311,Lung,Lung Neuroendocrine Tumor,Small Cell Lung Cancer,NaN,NaN,NaN,14.088938,10.298567,...,5.892877,6.482203,6.258330,6.425761,6.454340,6.050284,5.829088,6.321567,5.973841,6.723558
2,ACH-000695,CORL47,Lung,Lung Neuroendocrine Tumor,Small Cell Lung Cancer,NaN,NaN,NaN,14.802760,12.660535,...,5.856488,6.421728,5.928844,6.166514,6.454669,6.084489,5.831370,6.084489,5.831370,6.735522
3,ACH-000743,CORL95,Lung,Lung Neuroendocrine Tumor,Small Cell Lung Cancer,NaN,NaN,NaN,14.687715,12.203541,...,5.778997,6.364397,6.293885,6.181302,6.557043,6.219943,5.902315,6.464015,6.059182,6.779654
4,ACH-000594,DMS153,Lung,Lung Neuroendocrine Tumor,Small Cell Lung Cancer,NaN,NaN,NaN,15.371449,13.265271,...,5.906890,6.691115,6.196332,6.455821,7.594922,6.106223,5.824513,6.380764,6.058966,7.831497


In [24]:
data['lineage'] = np.where(data['lineage_2'].str.contains('Non-small', case=False), 'Non-Small Cell', 'Small Cell')

In [25]:
data['lineage'].value_counts()

,count
lineage,
Non-Small Cell,119
Small Cell,49


In [26]:
columns_to_remove = ['depmap_id', 'cell_line_display_name', 'lineage_1', 'lineage_2', 'lineage_3', 'lineage_5', 'lineage_6', 'lineage_4']
data = data.drop(columns=columns_to_remove)

In [27]:
data.head(2)

,MIMAT0000062,MIMAT0000063,MIMAT0000064,MIMAT0000065,MIMAT0000066,MIMAT0000067,MIMAT0000414,MIMAT0000415,MIMAT0000416,MIMAT0000098,...,MIMAT0002191,MIMAT0002190,MIMAT0002189,MIMAT0002188,MIMAT0002187,MIMAT0002186,MIMAT0002185,MIMAT0010151,MIMAT0010150,lineage
0,14.735474,12.091720,9.961391,12.626818,9.808289,12.016231,12.647656,12.811913,6.214513,6.262471,...,6.609991,6.308885,6.262471,6.397461,5.966938,5.907852,6.572435,6.230741,6.910133,Small Cell
1,14.088938,10.298567,9.188911,9.433690,7.415911,9.548706,10.592279,10.094434,6.192391,6.209064,...,6.482203,6.258330,6.425761,6.454340,6.050284,5.829088,6.321567,5.973841,6.723558,Small Cell


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Columns: 735 entries, MIMAT0000062 to lineage
dtypes: float64(734), object(1)
memory usage: 964.8+ KB


In [29]:
# Check if any cell contains NaN
any_nan = data.isna().any().any()
any_nan

False

In [30]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [37]:
X_train.shape, X_train.dtype

((134, 734), dtype('float64'))

In [36]:
X_train[:10]

array([[ 2.09714814,  1.56102329,  1.64925314, ...,  0.65113048,
         2.07554345,  0.74653646],
       [-1.49864366, -0.68528856, -1.42676897, ..., -1.56292539,
        -0.96481616, -1.19661364],
       [-0.76953996, -1.85293162, -1.35101768, ..., -1.41050248,
         0.58077575,  0.84678714],
       ...,
       [ 0.01221754, -0.15509051, -0.13237277, ..., -0.33038845,
         0.52377647, -0.34580558],
       [ 0.31823448,  0.38752467,  0.58896584, ...,  1.09807715,
         1.13991008, -0.46742356],
       [ 1.04447861,  1.21311262,  0.91073568, ..., -0.30223258,
        -1.21984152,  1.44315884]])

In [38]:
y[:10], y.dtype

(0        Small Cell
 1        Small Cell
 2        Small Cell
 3        Small Cell
 4        Small Cell
 5        Small Cell
 6    Non-Small Cell
 7    Non-Small Cell
 8    Non-Small Cell
 9    Non-Small Cell
 Name: lineage, dtype: object,
 dtype('O'))

## Convert data to tensors and move to device

In [43]:
batch_size = 32

In [44]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor((y_train == 'Non-Small Cell').astype(int).values, dtype=torch.float32).reshape(-1, 1).to(device)
y_test_tensor = torch.tensor((y_test == 'Non-Small Cell').astype(int).values, dtype=torch.float32).reshape(-1, 1).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Define a Model

In [45]:
class BinaryClassificationModel(nn.Module):
    def __init__(self, input_dim, hidden_layers, dropout_rate):
        super(BinaryClassificationModel, self).__init__()

        layers = []
        last_dim = input_dim
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(last_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            last_dim = hidden_dim

        layers.append(nn.Linear(last_dim, 1))
        layers.append(nn.Sigmoid())

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

## Training Function

In [46]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=25):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")


## Evaluation Function

In [49]:
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            all_preds.extend(outputs.cpu().round().numpy())  # Moving predictions to CPU for scoring
    return accuracy_score(y_test_tensor.cpu(), all_preds)

In [51]:
import torch.optim as optim

In [52]:
input_dim = X_train.shape[1]
hyperparams = {
    'hidden_layers': [[64, 32], [128, 64, 32], [128, 64]],  # Number of neurons per layer
    'dropout_rate': [0.3, 0.4, 0.5],
    'learning_rate': [0.001, 0.0005],
    'optimizer': [optim.Adam, optim.SGD]
}


In [53]:
best_accuracy = 0.0
best_params = {}

In [54]:
for hidden_layers in hyperparams['hidden_layers']:
    for dropout_rate in hyperparams['dropout_rate']:
        for lr in hyperparams['learning_rate']:
            for opt_func in hyperparams['optimizer']:

                # Initialize model, criterion, and optimizer
                model = BinaryClassificationModel(input_dim, hidden_layers, dropout_rate).to(device)
                criterion = nn.BCELoss()
                optimizer = opt_func(model.parameters(), lr=lr)

                # Train and evaluate
                train_model(model, train_loader, criterion, optimizer, num_epochs=25)
                accuracy = evaluate_model(model, test_loader)

                # Check for best accuracy
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {
                        'hidden_layers': hidden_layers,
                        'dropout_rate': dropout_rate,
                        'learning_rate': lr,
                        'optimizer': opt_func
                    }

print("Best Parameters:", best_params)
print("Best Accuracy:", best_accuracy)

Epoch 1/25, Loss: 0.6484
Epoch 2/25, Loss: 0.5363
Epoch 3/25, Loss: 0.4598
Epoch 4/25, Loss: 0.3527
Epoch 5/25, Loss: 0.2848
Epoch 6/25, Loss: 0.2231
Epoch 7/25, Loss: 0.1775
Epoch 8/25, Loss: 0.1719
Epoch 9/25, Loss: 0.1275
Epoch 10/25, Loss: 0.0976
Epoch 11/25, Loss: 0.0923
Epoch 12/25, Loss: 0.0709
Epoch 13/25, Loss: 0.0732
Epoch 14/25, Loss: 0.0486
Epoch 15/25, Loss: 0.0544
Epoch 16/25, Loss: 0.0515
Epoch 17/25, Loss: 0.0260
Epoch 18/25, Loss: 0.0365
Epoch 19/25, Loss: 0.0293
Epoch 20/25, Loss: 0.0311
Epoch 21/25, Loss: 0.0137
Epoch 22/25, Loss: 0.0137
Epoch 23/25, Loss: 0.0352
Epoch 24/25, Loss: 0.0103
Epoch 25/25, Loss: 0.0125
Epoch 1/25, Loss: 0.7065
Epoch 2/25, Loss: 0.7085
Epoch 3/25, Loss: 0.7066
Epoch 4/25, Loss: 0.7232
Epoch 5/25, Loss: 0.7125
Epoch 6/25, Loss: 0.7115
Epoch 7/25, Loss: 0.7014
Epoch 8/25, Loss: 0.7111
Epoch 9/25, Loss: 0.7188
Epoch 10/25, Loss: 0.7067
Epoch 11/25, Loss: 0.6957
Epoch 12/25, Loss: 0.7003
Epoch 13/25, Loss: 0.7000
Epoch 14/25, Loss: 0.6992
Epoc